In [2]:
!pip install dvc


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [88]:
import os
import subprocess
import pandas as pd

In [89]:
def execute(command):
    process = subprocess.run(command, shell=True, text=True, capture_output=True)
    print(process.stdout)
    

In [90]:
def initialize_repo():
    #os.makedirs(repo_path, exist_ok=True)
    #execute(f'cd {repo_path}')
    execute('git init')
    execute('dvc init')
    execute('dvc config core.autostage true')

In [91]:
def add_file(file):
    # add to dvc
    execute(f"dvc add {file}")
    # commit changes to git
    execute(f"git add {file}.dvc .gitignore")
    execute(f"git commit -m 'Add {file} to DVC'")

In [92]:
def add_version(file):
    # add to dvc
    execute(f"dvc add {file}")
    # commit changes to git
    execute(f"git add {file}.dvc")
    execute(f"git commit -m 'Add {file} version to DVC'")

In [93]:
def get_version(file):
    execute(f"git checkout HEAD^1 {file}")
    execute('dvc checkout')

In [101]:
def revert_version(file):
    execute(f"git commit {file}.dvc -m 'Revert file'")

In [94]:
def process_file(file_path):
    # read csv file
    df = pd.read_csv(file_path)
    
    # concatenate the addressline1 and addressline2 and save them in addressline
    df = df.assign(ADDRESSLINE1 = df.apply(lambda x: x['ADDRESSLINE1'] if pd.isnull(x['ADDRESSLINE2']) else f"{x['ADDRESSLINE1']}, {x['ADDRESSLINE2']}",axis=1))
    df = df.drop('ADDRESSLINE2', axis = 1)
    df = df.rename(columns={'ADDRESSLINE1':'ADDRESSLINE'})
    
    # null values of states are not modified as only Australia and USA have states
    # set the TERRITORY of USA and Canada to be NA
    df.loc[df['COUNTRY'] == 'USA', 'TERRITORY'] = 'NA'
    df.loc[df['COUNTRY'] == 'Canada', 'TERRITORY'] = 'NA'
    
    # null values of postal codes are not fixed due to missing information from the customere
    # save back to csv
    df.to_csv(file_path, index=False)

In [95]:
initialize_repo()

Reinitialized existing Git repository in /Users/layalsaadeddine/Desktop/FSD 2/ML/.git/





In [96]:
file_path = "Customers.csv"
add_file(file_path)



On branch master
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .DS_Store
	modified:   .dvc/config
	modified:   DVC_initialization.ipynb
	deleted:    NewCustomers.csv.dvc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/DVC_local-checkpoint.ipynb
	DVC_local.ipynb

no changes added to commit (use "git add" and/or "git commit -a")



In [97]:
process_file(file_path)

In [98]:
add_version(file_path)



[master e4b0617] Add Customers.csv version to DVC
 1 file changed, 2 insertions(+), 2 deletions(-)



In [99]:
get_version("Customers.csv.dvc")

In [100]:
execute("ls -lh")

total 456
-rw-r--r--@ 1 layalsaadeddine  staff    97K Jul 30 14:39 Customers.csv
-rw-r--r--  1 layalsaadeddine  staff    95B Jul 30 14:58 Customers.csv.dvc
-rw-r--r--  1 layalsaadeddine  staff    43K Jul 30 14:03 DVC_initialization.ipynb
-rw-r--r--  1 layalsaadeddine  staff    23K Jul 30 14:57 DVC_local.ipynb
-rw-r--r--  1 layalsaadeddine  staff    52K Jul 30 11:16 Untitled.ipynb

